In [54]:
import os
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import radiomics
import itertools
import warnings
from glob import glob
import radiomics
import cv2

# ImageProcessing

This notebook is dedicated to performing image processing tasks that are mainly applied before feature extraction to provide meaningful images that result in reproducible, and interpretable feature space. The tasks implemented here are as follows:

- Histogram matching
- Segmentation
- Filtered image normalization

The tasks mainly make use of ImageProcessing class from tmaR module. 

In [8]:
writer = sitk.ImageFileWriter()
binarizer = sitk.BinaryThresholdImageFilter()
multiply = sitk.MultiplyImageFilter()

rescaler = sitk.RescaleIntensityImageFilter()
rescaler.SetOutputMinimum(0)  # 8-bit minimum
rescaler.SetOutputMaximum(255)  # 8-bit maximum

## Histogram Matching

Laszlo G. Nyul, Jayaram K. Udupa, and Xuan Zhang, "New Variants of a Method of MRI Scale Standardization", IEEE Transactions on Medical Imaging, 19(2):143-150, 2000.
<br>
https://simpleitk.org/doxygen/latest/html/classitk_1_1simple_1_1HistogramMatchingImageFilter.html



In [4]:
HistMatch = sitk.HistogramMatchingImageFilter()

# Set parameters
HistMatch.SetNumberOfHistogramLevels(256)  # Number of bins in the histogram
HistMatch.SetNumberOfMatchPoints(50)       # Number of points to use for matching
HistMatch.ThresholdAtMeanIntensityOn() # Whether to threshold at mean intensity

In [ ]:
os.chdir('') # path to the folder
ImageNames = glob('*.tif') # all the tif files are read for histogram template matching 
ImageNames.remove("Reference.tif") # there must be an image in the folder with Reference.tif name as the reference for the process
ReferenceImage = sitk.ReadImage('Reference.tif')
for fid in ImageNames:
    InputImage = sitk.ReadImage(fid)
    OutputImage = HistMatch.Execute(InputImage,ReferenceImage)
    writer.SetFileName(f"Proc_{fid}")
    writer.Execute(OutputImage)
    del OutputImage, InputImage

## Filtered image normalization

Here we have functions that automatically normalize filtered images that were failed to normalize with the pyradiomics library internal code. And as a results discretized images are **informationless**.
This is because of embedding artifacts associated with the TMAs, are not commonly encountered where pyradiomics or other libraries are used. There exist several filters we are using for feature extraction. We divide these filters to two categories, 1 - Immune and 2- prone to normalization problem:<br>

**Immune:**
- **Original:** This image is manually normalized as said above in histogram matching section.
- **Exponential:** Due to its nature it increases contrast and no normalization is needed.  
- **Square:** Due to its nature it increases contrast and no normalization is needed.  
- **LBP3D:** It is a texture enconding algorithm and does not make sense to normalized it.

<br> **Prone:**
- **Gradient:** This highlights variations in the image and since the air holes and embedding artifacts are most abrupt change in the images, they tend to dominate higher gray values, as a result, phenotypical charectristics are dumped into a narrow span of dynamic range and in discretization infornation loss is considerable.
- **LoG:** Same as Gradient filter.
- **Wavelet:** Same as Gradient filter.
- Logarithm
- SquareRoot

In [ ]:
os.chdir('')
binary = sitk.ReadImage('mask.tif', imageIO="TIFFImageIO")

binarizer.SetUpperThreshold(65535)
binarizer.SetLowerThreshold(1)
binary = binarizer.Execute(binary)
binary = sitk.Cast(binary, sitk.sitkUInt8)

stack = sitk.ReadImage('image.tif', imageIO="TIFFImageIO")

### Active Filters

#### 1- LoG

Laplacian of Gaussian filter, edge enhancement filter. Emphasizes areas of gray level change, where sigma defines how coarse the emphasised texture should be. A low sigma emphasis on fine textures (change over a short distance), where a high sigma value emphasises coarse textures (gray level change over a large distance).

In [35]:
for Image, ImageName, inputKwargs in radiomics.imageoperations.getLoGImage(stack, binary,sigma=[2]):
    writer.SetFileName(f"{ImageName}.tif")
    Image = rescaler.Execute(Image)
    writer.Execute(sitk.Cast(Image,sitk.sitkUInt8))
    del Image

#### 2- Wavelet
Wavelet filtering, yields 8 decompositions per level (all possible combinations of applying either a High or a Low pass filter in each of the three dimensions.

For RAM usage optimization we don't have a separate block for each step of generation, processing, and writing of wavelet transform.

In [36]:
# Rescale intensity to 8-bit
for DecompImage, DecompName, inputKwargs in radiomics.imageoperations.getWaveletImage(stack, binary, wavelet='haar'):
    DecompImage = rescaler.Execute(DecompImage)
    DecompImage = sitk.Cast(DecompImage,sitk.sitkUInt8)
    writer.SetFileName(DecompName+'.tif')
    writer.Execute(DecompImage)
    del DecompImage

### Passive Filters

#### 1- Logarithm
takes the logarithm of the absolute intensity + 1. Values are scaled to original range and negative original values are made negative again after application of filter.

In [37]:
for Image, ImageName, inputKwargs in radiomics.imageoperations.getLogarithmImage(stack, binary):
    writer.SetFileName(f"{ImageName}.tif")
    Image = rescaler.Execute(Image)
    writer.Execute(sitk.Cast(Image,sitk.sitkUInt8))

#### 2- SquareRoot
Takes the square root of the absolute image intensities and scales them back to original range. Negative values in the original image will be made negative again after application of filter.

In [38]:
for Image, ImageName, inputKwargs in radiomics.imageoperations.getSquareRootImage(stack, binary):
    writer.SetFileName(f"{ImageName}.tif")
    Image = rescaler.Execute(Image)
    writer.Execute(sitk.Cast(Image,sitk.sitkUInt8))

#### 3- Gradient
Returns the gradient magnitude.

In [39]:
for Image, ImageName, inputKwarg in radiomics.imageoperations.getGradientImage(stack, binary):
    writer.SetFileName(f"{ImageName}.tif")
    Image = rescaler.Execute(Image)
    writer.Execute(sitk.Cast(Image,sitk.sitkUInt8))

#### 4- Exponential
Takes the the exponential, where filtered intensity is e to the power of absoluteintensity. Values are scaled to original
range and negative original values are made negative again after application of filter.

In [52]:
for Image, ImageName, inputKwargs in radiomics.imageoperations.getExponentialImage(stack, binary):
    writer.SetFileName(f"{ImageName}.tif")
    Image = rescaler.Execute(Image)
    writer.Execute(sitk.Cast(Image,sitk.sitkUInt8))

#### 5- Square
Takes the square of the image intensities and linearly scales them back to the original range. Negative values in79
the original image will be made negative again after application of filter.80

In [53]:
for iMAGE, ImageName, inputKwargs in radiomics.imageoperations.getSquareImage(stack, binary):
    writer.SetFileName(f"{ImageName}.tif")
    Image = rescaler.Execute(Image)
    writer.Execute(sitk.Cast(Image,sitk.sitkUInt8))

## Histogram Stretching

In [ ]:
path = '' # parent folder with sub folders including the images to be stretched
files = os.walk(path) 
name = 'wavelet-HLH.tif' # name of the filter/image to be stretched. 
writer = sitk.ImageFileWriter()
Operator = sitk.IntensityWindowingImageFilter()
minimum, maximum = [30,220] # Minimum and maximum of the window used for stretching the gray values

for root, dirs, files in files:
    os.chdir(root)
    if any(name in x for x in files) == True:
        image = sitk.ReadImage(name, imageIO="TIFFImageIO")
        image = sitk.IntensityWindowing(image,windowMinimum=minimum,windowMaximum=maximum,outputMinimum=0,outputMaximum=255)
        writer.SetFileName(name)
        writer.Execute(image)
    else:
        warnings.warn(f"{name} does not exist in this root")

## Discretitazion
This code block is developed to visualize what happens in dicretization and provide an intution of what discretization is doing.

In [8]:
path = '' # path to the folder containing the image
os.chdir(path)
name = 'image.tif' # name of the image to discritize
writer = sitk.ImageFileWriter()
iMAGE = sitk.ReadImage(name)
iMAGE_array = sitk.GetArrayFromImage(iMAGE)
bin_number = 16
discrete=cv2.normalize(iMAGE_array, None, 0, bin_number,cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)
discrete = sitk.GetImageFromArray(discrete)
writer.SetFileName(f"{bin_number}Bins.tif")
writer.Execute(discrete)